# Transforms

> A set of transforms for point cloud data augmentation

In [ ]:
#| default_exp transforms

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import numpy as np
import torch

In [ ]:
#| export
class Transform:
    '''A base class for transforms. In case a tuple or a list is the input it will only apply the transforms to the first element and return the others as they are.
       At children class you have to define a forward method that will perform the transform'''
    def __call__(self, x):
        if not isinstance(x, (tuple, list)): x = [x]
        x, *args = x
        x = self.forward(x)
        
        if len(args) > 0:
            if not isinstance(x, (tuple, list)): x = [x]
            x = x + args
        
        return x
    
    def forward(self, x):
        raise NotImplementedError

In [ ]:
#| export 
class FirstKPointsKeep(Transform):
    "Keep a subset of the original pointcloud containing the K first points"
    def __init__(self, num_points:int): # number of points to keep
        self.num_points = num_points

    def forward(self, x):
        return x[:self.num_points, :]

In [ ]:
#| hide
t = FirstKPointsKeep(512)
pc = np.random.random((1024, 3))
pc2 = t(pc)
print(f'Shape:: Before transform: {pc.shape} | After transform: {pc2.shape}')

Shape:: Before transform: (1024, 3) | After transform: (512, 3)


In [ ]:
#| export 
class RandomPointKeep(Transform):
    "Keep a random subset of points from the original pointcloud"
    def __init__(self, num_points:int): # number of points to keep
        self.num_points = num_points
        
    def forward(self, x):
        np.random.shuffle(x)
        return x[:self.num_points, :]

In [ ]:
#| hide
t = RandomPointKeep(512)
pc = np.random.random((1024, 3))
pc2 = t(pc)
print(f'Shape:: Before transform: {pc.shape} | After transform: {pc2.shape}')

Shape:: Before transform: (1024, 3) | After transform: (512, 3)


In [ ]:
#| export 
class UnitSphereNormalization(Transform):
    "Normalizes the coordinates of the point cloud, so that all points are inside the unit sphere"
    def __init__(self, return_scale=False): self.return_scale=return_scale
    def forward(self, pc):
        centroid = np.mean(pc, axis=0)
        pc = pc - centroid
        m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
        pc = pc / m
        if self.return_scale: pc = (pc, m)
        return pc

In [ ]:
#| export 
class RandomShuffle(Transform):
    "Shuffles the order of the points inside the point cloud"
    def forward(self, pc):
        np.random.shuffle(pc)
        return pc

In [ ]:
#| export 
class AnisotropicScale(Transform):
    "Performs a random anisotropic scaling of the point cloud"
    def forward(self, pc):
        xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
        xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
        pointcloud = pc
        translated_pointcloud = pointcloud * xyz1 + xyz2
        return translated_pointcloud.astype(np.float32)

In [ ]:
#| export 
class RandomPointDropout(Transform):
    "Randomly removes a subset of the point cloud. To keep the shape of the point cloud constant we set all the removed points to have the same coordinates."
    def __init__(self, max_dropout_ratio = 0.875):
        self.max_dropout_ratio = max_dropout_ratio
        
    def forward(self, pc):  
        dropout_ratio = np.random.random() * self.max_dropout_ratio # 0 ~ 0.875
        drop_idx = np.where(np.random.random((pc.shape[0])) <= dropout_ratio)[0]

        if len(drop_idx) > 0:
            pc[drop_idx,:] = pc[0,:] # set to the first point

        return pc

In [ ]:
#| export 
class ToTensor(Transform):
    "Turns a numpy array to a torch tensor"
    def forward(self, pc):
        pc = torch.from_numpy(pc)
        return pc
